In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf

In [ ]:
data = np.load("conv2d_daily.npy")
data = data[1:, :, :]
data.shape

In [ ]:
year_info = pd.read_excel("../year_info.xlsx")
year_info.shape

Start from:

In [ ]:
year_info[year_info["year"] < 1975].shape

In [ ]:
year_info[year_info["year"] < 1975]["is_leap"].sum()

In [ ]:
(74-18)*365 + 18*366

End:

In [ ]:
year_info[year_info["year"] >= 1975][year_info["year"] <= 2000].shape

In [ ]:
year_info[year_info["year"] >= 1975][year_info["year"] <= 2000]["is_leap"].sum()

In [ ]:
(26-7)*365 + 7*366

In [ ]:
data_1975_2000 = data[27028: 27028+9497]
data_1975_2000.shape

In [ ]:
dataset = np.expand_dims(data_1975_2000, axis=-1)
dataset = np.expand_dims(dataset, axis=0)

dataset.shape

In [ ]:
# def create_shifted_frames(data):
#     x = data[0 : data.shape[0] - 1, :, :]
#     y = data[ 1 : data.shape[0], :, :]
#     return x, y

def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    y = data[:, 1 : data.shape[1], :, :]
    return x, y

In [ ]:
x_train, y_train = create_shifted_frames(dataset)
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))

In [ ]:
shape=(None, *x_train.shape[2:])
shape

In [ ]:
# Construct the input layer with no definite frame size.
inp = tf.keras.layers.Input(shape=(None, *x_train.shape[2:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = tf.keras.layers.ConvLSTM2D(
    filters=64,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ConvLSTM2D(
    filters=64,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = tf.keras.layers.Conv3D(
    filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
)(x)

# Next, we will build the complete model and compile it.
model = tf.keras.models.Model(inp, x)
model.compile(
    loss=tf.keras.losses.binary_crossentropy, optimizer=tf.keras.optimizers.Adam(),
)

In [ ]:
# Define modifiable training hyperparameters.
epochs = 5
batch_size = 5

# Fit the model to the training data.
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
)